In [1]:
import chainer

/home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
import chainer.functions as F

In [3]:
import chainer.links as L

In [4]:
from chainer import training, datasets, iterators, optimizers

In [5]:
from chainer.training import extensions

In [6]:
import numpy as np

In [7]:
batch_size = 10

In [8]:
uses_device = 0

In [9]:
class NMIST_Conv_NN(chainer.Chain):

        def __init__(self):
                super(NMIST_Conv_NN, self).__init__()
                with self.init_scope():
                        self.conv1 = L.Convolution2D(1, 8, ksize=3)     # フィルタサイズ＝3で出力数8
                        self.linear1 = L.Linear(1352, 10)                       # 出力数10

        def __call__(self, x, t=None, train=True):
                # 畳み込みニューラルネットワークによる画像認識
                h1 = self.conv1(x)                              # 畳み込み層
                h2 = F.relu(h1)                                 # 活性化関数
                h3 = F.max_pooling_2d(h2, 2)# プーリング層
                h4 = self.linear1(h3)                   # 全結合層
                # 損失か結果を返す
                return F.softmax_cross_entropy(h4, t) if train else F.softmax(h4)

In [10]:
# ニューラルネットワークを作成
model = NMIST_Conv_NN()

In [11]:
if uses_device >= 0:
        # GPUを使う
        chainer.cuda.get_device_from_id(0).use()
        chainer.cuda.check_cuda_available()
        # GPU用データ形式に変換
        model.to_gpu()

In [12]:
# MNISTデータセットを用意する
train, test = chainer.datasets.get_mnist(ndim=3)

In [13]:
# 繰り返し条件を作成する
train_iter = iterators.SerialIterator(train, batch_size, shuffle=True)
test_iter = iterators.SerialIterator(test, batch_size, repeat=False, shuffle=False)

In [14]:
# 誤差逆伝播法アルゴリズムを選択する
optimizer = optimizers.Adam()
optimizer.setup(model)

In [15]:
# デバイスを選択してTrainerを作成する
updater = training.StandardUpdater(train_iter, optimizer, device=uses_device)
trainer = training.Trainer(updater, (5, 'epoch'), out="result")

In [16]:
# テストをTrainerに設定する
trainer.extend(extensions.Evaluator(test_iter, model, device=uses_device))

In [17]:
# 学習の進展を表示するようにする
trainer.extend(extensions.ProgressBar())

In [18]:
# 機械学習を実行する
trainer.run()

     total [..................................................]  0.33%
this epoch [..................................................]  1.67%
       100 iter, 0 epoch / 5 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  0.67%
this epoch [#.................................................]  3.33%
       200 iter, 0 epoch / 5 epochs
    270.23 iters/sec. Estimated time to finish: 0:01:50.278154.
     total [..................................................]  1.00%
this epoch [##................................................]  5.00%
       300 iter, 0 epoch / 5 epochs
    277.48 iters/sec. Estimated time to finish: 0:01:47.033903.
     total [..................................................]  1.33%
this epoch [###...............................................]  6.67%
       400 iter, 0 epoch / 5 epochs
    280.46 iters/sec. Estimated time to finish: 0:01:45.540699.
     total [...........................

     total [#####.............................................] 11.33%
this epoch [############################......................] 56.67%
      3400 iter, 0 epoch / 5 epochs
    286.59 iters/sec. Estimated time to finish: 0:01:32.816290.
     total [#####.............................................] 11.67%
this epoch [#############################.....................] 58.33%
      3500 iter, 0 epoch / 5 epochs
    286.61 iters/sec. Estimated time to finish: 0:01:32.460714.
     total [######............................................] 12.00%
this epoch [##############################....................] 60.00%
      3600 iter, 0 epoch / 5 epochs
     286.6 iters/sec. Estimated time to finish: 0:01:32.113669.
     total [######............................................] 12.33%
this epoch [##############################....................] 61.67%
      3700 iter, 0 epoch / 5 epochs
    285.83 iters/sec. Estimated time to finish: 0:01:32.011678.
     total [######..............

     total [###########.......................................] 22.33%
this epoch [#####.............................................] 11.67%
      6700 iter, 1 epoch / 5 epochs
    267.47 iters/sec. Estimated time to finish: 0:01:27.112142.
     total [###########.......................................] 22.67%
this epoch [######............................................] 13.33%
      6800 iter, 1 epoch / 5 epochs
    267.73 iters/sec. Estimated time to finish: 0:01:26.654582.
     total [###########.......................................] 23.00%
this epoch [#######...........................................] 15.00%
      6900 iter, 1 epoch / 5 epochs
    267.97 iters/sec. Estimated time to finish: 0:01:26.202517.
     total [###########.......................................] 23.33%
this epoch [########..........................................] 16.67%
      7000 iter, 1 epoch / 5 epochs
    268.21 iters/sec. Estimated time to finish: 0:01:25.752366.
     total [###########.........

     total [################..................................] 33.33%
this epoch [#################################.................] 66.67%
     10000 iter, 1 epoch / 5 epochs
    273.21 iters/sec. Estimated time to finish: 0:01:13.203286.
     total [################..................................] 33.67%
this epoch [##################################................] 68.33%
     10100 iter, 1 epoch / 5 epochs
    273.33 iters/sec. Estimated time to finish: 0:01:12.805044.
     total [#################.................................] 34.00%
this epoch [###################################...............] 70.00%
     10200 iter, 1 epoch / 5 epochs
    273.49 iters/sec. Estimated time to finish: 0:01:12.398679.
     total [#################.................................] 34.33%
this epoch [###################################...............] 71.67%
     10300 iter, 1 epoch / 5 epochs
    273.49 iters/sec. Estimated time to finish: 0:01:12.030870.
     total [#################...

     total [######################............................] 44.33%
this epoch [##########........................................] 21.67%
     13300 iter, 2 epoch / 5 epochs
    261.73 iters/sec. Estimated time to finish: 0:01:03.806745.
     total [######################............................] 44.67%
this epoch [###########.......................................] 23.33%
     13400 iter, 2 epoch / 5 epochs
    261.72 iters/sec. Estimated time to finish: 0:01:03.427465.
     total [######################............................] 45.00%
this epoch [############......................................] 25.00%
     13500 iter, 2 epoch / 5 epochs
    261.71 iters/sec. Estimated time to finish: 0:01:03.048056.
     total [######################............................] 45.33%
this epoch [#############.....................................] 26.67%
     13600 iter, 2 epoch / 5 epochs
     261.7 iters/sec. Estimated time to finish: 0:01:02.667798.
     total [####################

     total [###########################.......................] 55.33%
this epoch [######################################............] 76.67%
     16600 iter, 2 epoch / 5 epochs
    273.09 iters/sec. Estimated time to finish: 0:00:49.067844.
     total [###########################.......................] 55.67%
this epoch [#######################################...........] 78.33%
     16700 iter, 2 epoch / 5 epochs
    273.08 iters/sec. Estimated time to finish: 0:00:48.702981.
     total [###########################.......................] 56.00%
this epoch [#######################################...........] 80.00%
     16800 iter, 2 epoch / 5 epochs
    273.08 iters/sec. Estimated time to finish: 0:00:48.336994.
     total [############################......................] 56.33%
this epoch [########################################..........] 81.67%
     16900 iter, 2 epoch / 5 epochs
    273.08 iters/sec. Estimated time to finish: 0:00:47.970544.
     total [####################

     total [#################################.................] 66.33%
this epoch [###############...................................] 31.67%
     19900 iter, 3 epoch / 5 epochs
    261.53 iters/sec. Estimated time to finish: 0:00:38.618804.
     total [#################################.................] 66.67%
this epoch [################..................................] 33.33%
     20000 iter, 3 epoch / 5 epochs
    261.73 iters/sec. Estimated time to finish: 0:00:38.207814.
     total [#################################.................] 67.00%
this epoch [#################.................................] 35.00%
     20100 iter, 3 epoch / 5 epochs
    261.73 iters/sec. Estimated time to finish: 0:00:37.825464.
     total [#################################.................] 67.33%
this epoch [##################................................] 36.67%
     20200 iter, 3 epoch / 5 epochs
    261.73 iters/sec. Estimated time to finish: 0:00:37.443436.
     total [####################

     total [######################################............] 77.33%
this epoch [###########################################.......] 86.67%
     23200 iter, 3 epoch / 5 epochs
    272.75 iters/sec. Estimated time to finish: 0:00:24.931507.
     total [######################################............] 77.67%
this epoch [############################################......] 88.33%
     23300 iter, 3 epoch / 5 epochs
     272.7 iters/sec. Estimated time to finish: 0:00:24.568871.
     total [#######################################...........] 78.00%
this epoch [############################################......] 90.00%
     23400 iter, 3 epoch / 5 epochs
     272.7 iters/sec. Estimated time to finish: 0:00:24.202440.
     total [#######################################...........] 78.33%
this epoch [#############################################.....] 91.67%
     23500 iter, 3 epoch / 5 epochs
     272.7 iters/sec. Estimated time to finish: 0:00:23.835744.
     total [####################

     total [############################################......] 88.33%
this epoch [####################..............................] 41.67%
     26500 iter, 4 epoch / 5 epochs
     261.5 iters/sec. Estimated time to finish: 0:00:13.384540.
     total [############################################......] 88.67%
this epoch [#####################.............................] 43.33%
     26600 iter, 4 epoch / 5 epochs
     261.5 iters/sec. Estimated time to finish: 0:00:13.001960.
     total [############################################......] 89.00%
this epoch [######################............................] 45.00%
     26700 iter, 4 epoch / 5 epochs
    261.51 iters/sec. Estimated time to finish: 0:00:12.619251.
     total [############################################......] 89.33%
this epoch [#######################...........................] 46.67%
     26800 iter, 4 epoch / 5 epochs
    261.51 iters/sec. Estimated time to finish: 0:00:12.236626.
     total [####################

     total [#################################################.] 99.33%
this epoch [################################################..] 96.67%
     29800 iter, 4 epoch / 5 epochs
     272.9 iters/sec. Estimated time to finish: 0:00:00.732864.
     total [#################################################.] 99.67%
this epoch [#################################################.] 98.33%
     29900 iter, 4 epoch / 5 epochs
    272.91 iters/sec. Estimated time to finish: 0:00:00.366426.
     total [##################################################] 100.00%
this epoch [..................................................]  0.00%
     30000 iter, 5 epoch / 5 epochs
    261.66 iters/sec. Estimated time to finish: 0:00:00.


In [19]:
# 学習結果を保存する
chainer.serializers.save_hdf5( 'chapt02.hdf5', model )

# 結果を見る

In [23]:
# 学習結果を読み込む
chainer.serializers.load_hdf5( 'mnist1.hdf5', model )